# Laboratorio 7 - Deep Learning

## Autores

- Angel Higueros 20460
- Fredy Velasquez 201011



## Task 1 - Práctica
Considere las arquitecturas conversadas durante la clase, con ello realice una implementación de dos arquitecturas
usando PyTorch
1. Implemente la arquitectura de LeNet-5 para resolver el problema de clasificación del daset de dígitos escritos a mano llamado mnist dataset
2. Implemente la arquitectura de AlexNet para resolver el problema de clasificación usando el dataset de imagenes llamado CIFAR10 dataset.

Para cada implementación defina y justifique (dentro del notebook) una métrica de desempeño. Además responda
(en su notebook) 


Recuerde justificar y/o expandir su respuesta:
- ¿Cuál es la diferencia principal entre ambas arquitecturas?
- ¿Podría usarse LeNet-5 para un problema como el que resolvió usando AlexNet? ¿Y viceversa?
- Indique claramente qué le pareció más interesante de cada arquitectura




## Task 2 - Teoría
Responda claramente y con una extensión adecuada las siguientes preguntas:
1. Investigue e indique en qué casos son útiles las siguientes arquitecturas, agregue imagenes si esto le ayuda a una mejor comprensión
- GoogleNet (Inception)

La arquitectura GoogleNet es útil en casos donde se busca una red profunda con una computación eficiente. Su característica distintiva es el uso de módulos de "inception" que consisten en múltiples convoluciones y concatenaciones de características de diferentes tamaños de kernel en paralelo. Esto permite que la red capture características a diferentes escalas y resoluciones, lo que es útil en tareas de clasificación de imágenes, detección de objetos y segmentación semántica. GoogleNet es especialmente efectiva en la optimización de uso de recursos computacionales.

- DenseNet (Densely Connected Convolutional Networks)

DenseNet es útil cuando se busca una arquitectura que promueva conexiones densas entre capas. En lugar de tener conexiones dispersas entre capas, como en las CNN tradicionales, DenseNet conecta todas las capas entre sí. Esto fomenta el flujo de información a lo largo de la red y facilita el aprendizaje de características complejas y la mitigación del problema de desvanecimiento de gradientes. DenseNet es beneficioso para la clasificación de imágenes, detección de objetos y segmentación de imágenes.

- MobileNet

MobileNet es útil en casos donde se requiere una red ligera y eficiente en términos de recursos computacionales, como en aplicaciones móviles o dispositivos con recursos limitados. Esta arquitectura utiliza convoluciones separables en profundidad (depthwise separable convolutions) para reducir la cantidad de operaciones y parámetros, sin sacrificar demasiado la precisión. MobileNet es ideal para la clasificación de imágenes en dispositivos móviles y la detección en tiempo real.

- EfficientNet

EfficientNet es útil en situaciones donde se busca un equilibrio óptimo entre el rendimiento y la eficiencia computacional. Esta arquitectura utiliza un enfoque de búsqueda en escalas compuestas para encontrar el tamaño de modelo adecuado para una tarea dada, escalando la profundidad, el ancho y la resolución de la red de manera equilibrada. EfficientNet ha demostrado ser altamente eficiente en términos de rendimiento de precisión en una amplia gama de tareas de visión por computadora, incluida la clasificación de imágenes y la detección de objetos.

2. ¿Cómo la arquitectura de transformers puede ser usada para image recognition?

La arquitectura de Transformers, que se originó en el procesamiento de lenguaje natural (NLP), también se puede utilizar para el reconocimiento de imágenes. Esto se logra mediante la adaptación de la arquitectura Transformer original para el procesamiento de secuencias de texto a la tarea de procesar matrices de imágenes.

El enfoque principal para usar Transformers en el reconocimiento de imágenes se llama "Transformers para visión" o "Vision Transformers". La idea central es tratar una imagen como una secuencia de parches (patches), donde cada parche es una porción de la imagen. Cada parche se representa como un vector y se pasa a través de la arquitectura Transformer.

Usar Transformer con imagenes: 

División de la imagen en parches: La imagen se divide en una cuadrícula de parches, y cada parche se transforma en un vector de características.
Posición de codificación: Se agrega información de posición a los vectores de características de los parches, similar a cómo se manejan las secuencias en NLP.
Capas de atención multiatención: Se aplican capas de atención multiatención (self-attention) para permitir que los parches se relacionen entre sí.
Redes completamente conectadas: Después de las capas de atención, se utilizan capas de redes neuronales completamente conectadas para realizar la clasificación.
Esta arquitectura ha demostrado ser efectiva en una variedad de tareas de visión por computadora, incluido el reconocimiento de objetos, la detección de objetos y la segmentación semántica. Un ejemplo de implementación de Vision Transformer (ViT) en PyTorch se encuentra en la biblioteca "PyTorch Vision" de Hugging Face.
